# Data mining Project

In [7]:
import pandas as pd

In [ ]:
file_path = 'data.txt'

df = pd.read_csv(file_path, delimiter='\t', header=None)


columns = [
    "ID","Gender", "Age_Group", "Residence", "Education_Level", "Source_of_Income",
    "Marital_Status", "Smoked_Cigarettes", "Year_Diagnosed", "Surgical_Treatment",
    "Chemotherapy", "Radiotherapy", "Immunotherapy", "Molecular_targeted_Therapy",
    "Hospitalization_Number", "Time_to_Treatment", "Medical_Treatment_Need",
    "Emotional_Impact", "Travel_Impact", "Quality_of_Life", "Symptoms_exp_cough",
    "Symptoms_exp_Hoarseness","Symptoms_exp_Blood_cough","Symptoms_exp_chestpain",
    "Symptoms_exp_Shortness_of_breath","Symptoms_exp_weakness","Symptoms_exp_None",
    "Symptom_Frequency", "Symptom_Household_Impact", "Sleep_Issues", "Support_From_Close",
    "Dependency_Fear", "Health_Satisfaction", "Daily_Life_Impact_physical","Daily_Life_Impact_Psychological",
    "Daily_Life_Impact_proffesional","Daily_Life_Impact_family_life","Daily_Life_Impact_social_life",
    "Daily_Life_Impact_no_effect","Energy_Level", "Self_Care", "Daily_Activities_Difficulty",
    "Work_Readiness", "Support_Satisfaction", "Coping_Strategy", "Negative_Emotions"
]

df.columns = columns
print(df)

df.to_csv('processed_survey_data_columns.csv', index=False)

      ID  Gender  Age_Group  Residence  Education_Level  Source_of_Income  \
0      1       1          1          2                4                 1   
1      2       1          2          4                3                 1   
2      3       1          2          2                4                 1   
3      4       2          4          4                3                 3   
4      5       2          3          1                2                 1   
..   ...     ...        ...        ...              ...               ...   
295  296       1          2          4                3                 1   
296  297       2          3          3                2                 1   
297  298       1          3          2                2                 1   
298  299       2          3          4                3                 1   
299  300       1          2          3                4                 4   

     Marital_Status  Smoked_Cigarettes  Year_Diagnosed  Surgical_Treatment 